In [ ]:
import serial  # type: ignore
import time
import mysql.connector  # type: ignore
from mysql.connector import Error  # type: ignore
from datetime import datetime

def connect_to_mysql():
    try:
        connection = mysql.connector.connect(
            host='localhost',
            database='arduino_uno',
            user='root',
            password='root'
        )
        if connection.is_connected():
            print("Conectado ao MySQL")
            return connection
    except Error as e:
        print(f"Erro ao conectar ao MySQL: {e}")
        return None

def insert_data(connection, valor):
    try:
        cursor = connection.cursor()
        query = "INSERT INTO dados_arduino (valor, data_hora) VALUES (%s, NOW())"
        cursor.execute(query, (valor,))
        connection.commit()
    except Error as e:
        print(f"Erro ao inserir dados: {e}")

def read_from_arduino(port, baudrate=9600, timeout=1):
    try:
        ser = serial.Serial(port, baudrate, timeout=timeout)
        time.sleep(2)
        print("Conexão estabelecida com o Arduino")
    except serial.SerialException as e:
        print(f"Erro ao conectar ao Arduino: {e}")
        return

    connection = connect_to_mysql()
    if not connection:
        ser.close()
        return

    last_insert_time = None

    try:
        while True:
            if ser.in_waiting > 0:
                line = ser.readline().decode('utf-8').rstrip()
                try:
                    sensor_value = int(line)
                    print(f"Valor do sensor: {sensor_value}")
                    current_datetime = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

                    valor_inserido = 1 if sensor_value > 200 else 0

                    if current_datetime != last_insert_time:
                        insert_data(connection, valor_inserido)
                        last_insert_time = current_datetime

                except ValueError:
                    print(f"Erro ao converter valor: {line}")
    except KeyboardInterrupt:
        print("Leitura interrompida pelo usuário")
    finally:
        ser.close()
        if connection.is_connected():
            connection.close()
            print("Conexão com MySQL fechada")
        print("Conexão serial fechada")

if __name__ == "__main__":
    port = "COM3"
    read_from_arduino(port)